In [1]:
from agents.llms.llm import LlamaCppChatCompletion
from agents.tools import get_current_weather, wikipedia_search, google_search, image_inspector
from rich import print

In [2]:
llm = LlamaCppChatCompletion.from_default_llm(n_ctx=0)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
llm.bind_tools([get_current_weather, google_search, wikipedia_search, image_inspector])

In [4]:
messages = [
    {"role": "system", "content": "You are a helpful assistant that has access to tools and use that to help humans."},
    {"role": "user", "content": "What is the weather in London?"}
]
output = llm.chat_completion(messages)
tool_results = llm.run_tool(output)
print(tool_results)

[
    {
        'tool_call_id': 'call_VTgSNCBzIRKcgbvVtc4Eqs32',
        'role': 'tool',
        'name': 'get_current_weather',
        'content': '{"FeelsLikeC": "20", "FeelsLikeF": "68", "cloudcover": "50", "humidity": "49", 
"localObsDateTime": "2024-05-08 02:58 PM", "observation_time": "01:58 PM", "precipInches": "0.0", "precipMM": 
"0.0", "pressure": "1029", "pressureInches": "30", "temp_C": "20", "temp_F": "68", "uvIndex": "5", "visibility": 
"10", "visibilityMiles": "6", "weatherCode": "116", "weatherDesc": [{"value": "Partly cloudy"}], "weatherIconUrl": 
[{"value": ""}], "winddir16Point": "SE", "winddirDegree": "140", "windspeedKmph": "7", "windspeedMiles": "4"}'
    }
]

In [9]:
image_url = "https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg"

messages = [
    {"role": "system", "content": "You are a helpful assistant that has access to tools and use that to help humans."},
    {"role": "user", "content": f"Check this image {image_url} and suggest me a location where I can go in London which looks similar"}
]
output = llm.chat_completion(messages)
tool_results = llm.run_tool(output)

updated_messages = messages + tool_results
messages = updated_messages + [{"role": "user", "content": "please answer me, based on the tool results."}]
output = llm.chat_completion(messages)
print(output.choices[0].message.content)

Based on the image you provided, it appears to be a nature boardwalk surrounded by lush green grass and a peaceful 
sky filled with clouds. The presence of people indicates that it could be a popular spot for outdoor activities or 
leisurely walks. If you're looking for a similar location in London, you might consider visiting one of the city's 
many parks or nature reserves. Here are a few suggestions:

1. **Richmond Park**: This is the largest royal park in London and offers a variety of landscapes, including open 
grasslands, wooded areas, and lakes. It's a great place for walking, cycling, and enjoying the outdoors.

2. **Hampstead Heath**: Another large green space in London, Hampstead Heath is known for its ponds, meadows, and 
woodlands. It's a popular spot for picnics, sunbathing, and hiking.

3. **Greenwich Park**: This park offers panoramic views of the city and is home to several historic buildings, 
including the Royal Observatory. It's a great place for a leisurely walk or a picnic.

4. **Victoria Park**: A smaller but still beautiful park in East London, Victoria Park is known for its lakes, 
gardens, and outdoor events.

5. **Hyde Park**: One of the most central parks in London, Hyde Park offers a variety of attractions, including the
Serpentine Lake, Speaker's Corner, and several monuments.

These locations all offer a peaceful and natural environment similar to the image you provided, making them 
excellent choices for a day out in London.

In [ ]:
messages = [
    {"role": "system", "content": "You are a helpful assistant that has access to tools and use that to help humans."},
    {"role": "user", "content": "Who won the 2024 mayor election in London?"}
]
output = llm.chat_completion(messages)
tool_results = llm.run_tool(output)

updated_messages = messages + tool_results
messages = updated_messages + [{"role": "user", "content": "please answer me, based on the tool results."}]
output = llm.chat_completion(messages)
print(output.choices[0].message.content)